In [1]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell

import numpy as np
import input_data

In [9]:
np.shape(trX[0])

(28, 28)

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28)
teX = teX.reshape(-1, 28, 28)

Extracting MNIST_data/train-images-idx3-ubyte.gz


/Users/K/anaconda/envs/tensorflow/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/K/Source/tensorflow_practice/input_data.py:41: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [9]:
input_vec_size = lstm_size = 28
time_step_size = 28

batch_size = 128
test_size = 256

In [6]:
def model(X, W, B, init_state, lstm_size):
    # X, input shape: (batch_size, input_vec_size, time_step_size)
    XT = tf.transpose(X, [1, 0, 2])  # permute time_step_size and batch_size
    # XT shape: (input_vec_size, batch_szie, time_step_size)
    XR = tf.reshape(XT, [-1, lstm_size]) # each row has input for each lstm cell (lstm_size)
    # XR shape: (input vec_size, batch_size)
    X_split = tf.split(0, time_step_size, XR) # split them to time_step_size (28 arrays)
    # Each array shape: (batch_size, input_vec_size)

    # Make lstm with lstm_size (each input vector size)
    lstm = rnn_cell.BasicLSTMCell(lstm_size, forget_bias=1.0)
    # Get lstm cell output, time_step_size (28) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = rnn.rnn(lstm, X_split, initial_state=init_state)

    return tf.matmul(outputs[-1], W) + B, lstm.state_size # State size to initialize the stat


In [10]:
W = init_weights([lstm_size, 10])
B = init_weights([10])

In [14]:
print (W)

In [12]:
py_x, state_size = model(trX[0], W, B, init_state, lstm_size)

ValueError: Shapes (3,) and (2,) are not compatible

In [2]:
# configuration
#                        O * W + b -> 10 labels for each image, O[? 28], W[28 10], B[10]
#                       ^ (O: output 28 vec from 28 vec input)
#                       |
#      +-+  +-+       +--+
#      |1|->|2|-> ... |28| time_step_size = 28
#      +-+  +-+       +--+
#       ^    ^    ...  ^
#       |    |         |
# img1:[28] [28]  ... [28]
# img2:[28] [28]  ... [28]
# img3:[28] [28]  ... [28]
# ...
# img128 or img256 (batch_size or test_size 256)
#      each input size = input_vec_size=lstm_size=28

# configuration variables
input_vec_size = lstm_size = 28
time_step_size = 28

batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, W, B, init_state, lstm_size):
    # X, input shape: (batch_size, input_vec_size, time_step_size)
    XT = tf.transpose(X, [1, 0, 2])  # permute time_step_size and batch_size
    # XT shape: (input_vec_size, batch_szie, time_step_size)
    XR = tf.reshape(XT, [-1, lstm_size]) # each row has input for each lstm cell (lstm_size)
    # XR shape: (input vec_size, batch_size)
    X_split = tf.split(0, time_step_size, XR) # split them to time_step_size (28 arrays)
    # Each array shape: (batch_size, input_vec_size)

    # Make lstm with lstm_size (each input vector size)
    lstm = rnn_cell.BasicLSTMCell(lstm_size, forget_bias=1.0)

    # Get lstm cell output, time_step_size (28) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = rnn.rnn(lstm, X_split, initial_state=init_state)
    print (outputs)
    print (_states)

    # Linear activation
    # Get the last output
    print (tf.matmul(outputs[-1], W) + B)
    return tf.matmul(outputs[-1], W) + B, lstm.state_size # State size to initialize the stat

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28)
teX = teX.reshape(-1, 28, 28)

# Tensorflow LSTM cell requires 2x n_hidden length (state & cell)
init_state = tf.placeholder("float", [None, 2*lstm_size])

X = tf.placeholder("float", [None, 28, 28])
Y = tf.placeholder("float", [None, 10])

# get lstm_size and output 10 labels
W = init_weights([lstm_size, 10])
B = init_weights([10])

py_x, state_size = model(X, W, B, init_state, lstm_size)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX), batch_size)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          init_state: np.zeros((batch_size, state_size))})

        test_indices = np.arange(len(teX))  # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         init_state: np.zeros((test_size, state_size))})))

Extracting MNIST_data/train-images-idx3-ubyte.gz


/Users/K/anaconda/envs/tensorflow/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/K/Source/tensorflow_practice/input_data.py:41: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
[<tf.Tensor 'RNN/BasicLSTMCell/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_1/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_2/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_3/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_4/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_5/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_6/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_7/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_8/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_9/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_10/mul_2:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'RNN/BasicLSTMCell_11/mul_2:0' shape=(?, 28) dtype=float32>, <t

KeyboardInterrupt: 